In [ ]:
## cv2 라이브러리 다운
# !pip install opencv-python 

In [ ]:
from openai import OpenAI
from getpass import getpass

from langchain.document_loaders import PyPDFLoader
import re
import olefile
import zlib
import struct

## PNG 파일에서 텍스트를 추출하고 요약하는 코드
import pytesseract
import cv2

In [9]:
def get_image_text(filename):
    # Tesseract-OCR 경로 설정
    pytesseract.pytesseract.tesseract_cmd = r"C:\\Users\\tjdrb\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe"

    # 이미지 로드
    img = cv2.imread(filename)
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # tesseract에서는 RGB로 넣어야 하기 때문에 변환

    # 이미지에서 텍스트 추출
    text = pytesseract.image_to_string(dst, lang='kor+eng')
    
    return text

# def extract_text_from_pdf(filename):
#     # PDF 파일 열기
#     pdf_document = fitz.open(filename)
#     text = ""

#     # 각 페이지에서 텍스트 추출
#     for page_num in range(len(pdf_document)):
#         page = pdf_document.load_page(page_num)
#         text += page.get_text()

#     return text.strip()
#### ---------------------------------------------------------------------- 수정 ------------------------------------------------------------------------------
def get_pdf_text(filename):

    # PyPDFLoader를 사용하여 PDF 파일 로드
    loader = PyPDFLoader(filename)
    documents = loader.load()

    # 전체 텍스트를 하나의 문자열로 저장
    text = "\n".join([doc.page_content for doc in documents])

    return text

def get_hwp_text(filename):
    # HWP 파일을 읽기 위한 olefile 객체 생성
    f = olefile.OleFileIO(filename)
    
    # HWP 파일 내의 디렉터리 목록을 가져옴
    dirs = f.listdir()

    # HWP 파일이 올바른 형식인지 확인 (FileHeader와 HwpSummaryInformation이 존재해야 함)
    if ["FileHeader"] not in dirs or \
            ["\x05HwpSummaryInformation"] not in dirs:
        raise Exception("Not Valid HWP.")

    # FileHeader 스트림을 열고 해당 데이터를 읽음
    header = f.openstream("FileHeader")
    header_data = header.read()

    # 파일이 압축되어 있는지 여부를 확인 (헤더의 36번째 바이트에 정보 있음)
    is_compressed = (header_data[36] & 1) == 1

    # BodyText 섹션의 번호를 찾기 위해 디렉토리에서 "Section"을 포함하는 항목 찾음
    nums = []
    for d in dirs:
        if d[0] == "BodyText":
            nums.append(int(d[1][len("Section"):]))

    # 섹션을 번호순으로 정렬하고 "BodyText/Section" 형식으로 리스트 생성
    sections = ["BodyText/Section" + str(x) for x in sorted(nums)]

    text = ""  # 최종적으로 추출될 텍스트 초기화

    # 각 섹션을 순차적으로 처리
    for section in sections:
        # 해당 섹션을 열고 데이터 읽기
        bodytext = f.openstream(section)
        data = bodytext.read()

        # 파일이 압축된 경우 압축을 해제
        if is_compressed:
            unpacked_data = zlib.decompress(data, -15)
        else:
            unpacked_data = data

        section_text = ""  # 각 섹션의 텍스트를 담을 변수
        i = 0
        size = len(unpacked_data)

        # 섹션 데이터에서 텍스트를 추출
        while i < size:
            # 각 레코드의 헤더를 읽고, 레코드 타입과 길이를 추출
            header = struct.unpack_from("<I", unpacked_data, i)[0]
            rec_type = header & 0x3ff  # 레코드 타입 추출
            rec_len = (header >> 20) & 0xfff  # 레코드 길이 추출

            # 레코드 타입이 67일 때 텍스트를 추출 (67은 텍스트 레코드 타입)
            if rec_type in [67]:
                rec_data = unpacked_data[i + 4:i + 4 + rec_len]
                section_text += rec_data.decode('utf-16')  # 텍스트를 UTF-16으로 디코딩
                section_text += "\n"  # 줄 바꿈 추가

            # 다음 레코드로 이동
            i += 4 + rec_len

        # 섹션의 텍스트를 전체 텍스트에 추가
        text += section_text
        text += "\n"  # 섹션 간에 줄 바꿈 추가

        text = re.sub(r'[^\x00-\x7F\uAC00-\uD7AF]', '', text)
        
    return text  # 최종적으로 추출된 텍스트 반환

In [ ]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {
                        '신청기간' : ['신청 기간', '접수 기간'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상', '신청 가능 지역'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항', '지역 제한'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '세부 심사 항목', '선정 기준'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용', '지원 한도'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의', '대출 취급 기관'],
                        '기타' : ['기타']
                        }

In [25]:

# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [ ]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = r'기업마당 공고문\[충남] 2025년 중소기업육성자금 지원계획 공고.pdf'

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('\\')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('\\')[1].split('.')[1]

# file_type이 pdf면 get_pdf_text, hwp면 get_hwp_text를 사용하여 텍스트 추출
if file_type == 'pdf':
    full_text = get_pdf_text(file_path)
elif file_type == 'hwp':
    full_text = get_hwp_text(file_path)
elif file_type == 'png':
    full_text = get_image_text(file_path)
# elif file_type == 'pdf':    ## pdf파일 image -> text 변환
#     full_text = extract_text_from_pdf(file_path)

RAG 사용???

In [31]:
base_template = {f'''
📌 **[{title}] 요약 안내**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**  

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**  

📢 **기타 안내**  
'''
}

In [ ]:
content = f'''당신은 파일 요약 전문가입니다. 사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 요약해야합니다.
파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니, 사전에 해당하는 단어가 나오면, 참고하여 빠짐 없이 요약해주십시오.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}
'''
user_content = f'''{full_text}를 요약해줘'''

completion = client.chat.completions.create(model='gpt-4o',
                                                messages=[{'role':'system','content':content},
                                                        {'role':'user','content':user_content}],
                                                        temperature=0)

print(completion.choices[0].message.content)

📌 **[[충남] 2025년 중소기업육성자금 지원계획 공고] 요약 안내**

📅 **신청기간**  
2025.1.8.(수)~자금소진시 까지 (한시자금), 2025.6.2.(월)~자금소진시 까지 (시군협력자금)

📝 **신청방법**  
충청남도에 사업장이 소재하는 중소기업 및 소상공인

🚫 **지원제외 대상**  
금융기관 불량거래처, 휴·폐업 기업, 매출액 기준 초과 기업, 자산총액 5천억 원 이상 기업, 상호출자 제한기업, 최근 5년간 정책자금 지원액 100억 원 초과 기업 등

⚠ **유의사항**  
대출금리는 시중금리 변동에 따라 변경 가능, 사업장 이전 시 이자 지원 중단, 자금 사용 상황 점검 가능

📑 **제출서류**  
사업자등록증, 재무제표, 사업장 확인서류, 지방세 과세증명서, 원천징수이행상황신고서 등

📌 **선정절차**  
신청 및 접수 → 위원회 심사 및 승인 → 융자추천 → 대출

💰 **지원내용**  
총 12,000억 원 규모, 창업 및 경쟁력 강화, 혁신형, 기업회생, 경영안정자금 등 다양한 자금 지원

📞 **문의처**  
충남경제진흥원 아산출장소(041-404-1482), 남부지소(041-404-1381), 충남신용보증재단(1588-7310)

📢 **기타 안내**  
힘쎈충남 금융지원센터 운영, 맞춤형 상품 안내, 금융지원센터 창구 및 전화번호 안내
